# Task 1 – The Dataset

All images are x-ray images taken in the chest region of children aged 1-5 from the
Guangzhou Women and Children’s Medical Center. There are 1100 images of healthy
humans, as well as 1100 images of people with pneumonia.
Keep in mind, that the image format is jpeg, and there are 3 color channels.
You need to organize the data into directories as shown on Figure 1. You need to determine
the training/validation/testing split yourselves but need to justify your split choice.

# Task 2 – Data Loading and Augmentation

We take the data from task 1, using the CustomDataset class we can load the data into the model, using torch DataLoader.
The data augmentation consists of a random horizontal flip, a random rotation up to 20 degrees and an affine transformation to make the model more robust for random patient oerientation. The randomaffine transform makes the model more robust. A scalejitter transformation makes the model more robust to images taken at different scales. Gaussian blur and noise make the model more robust to imperfect data.

# Task 3 – Construction the network

After successfully creating both the custom dataset and the dataloader, you need to create a
neural network, and use the data loader to feed the network. The architecture, complexity and
regularization are all up to you, but you need to justify your choices in comments. You are
more than welcome to replicate already known architectures or architectures we made during
the course, but you are NOT allowed to use any pretrained networks. You are also not
allowed to use any training data that is not included on ItsLearning.

Carefully consider which hyperparameters to test and strategically try to find the optimal
architecture for the task. In the comments, please describe your method for the optimization
and your choice of hyperparameters. Remember that there is an underlying competition, and
the highest accuracy wins. The competition will be measured based on the saved model, so
make sure to submit only the best one!

# Task 4 - Visualizing your results

Finally, you must visualize some aspects of your model. It can be a graph of the
training/validation performance, visualization of the filters or feature maps, or anything you
can think of. This must be saved as an image file and uploaded along with your model and
code.